In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
import seaborn as sns
sns.set_theme(context="notebook", font_scale=1.4,
              rc={"figure.figsize": [10, 6]})

In [ ]:
line = np.linspace(0, 8, 100)
train = 1./(1 + np.exp(-line))
plt.plot(train, label='training accuracy')
gen_true = - (line/ 10) ** 2 + (line/10 - .5) ** 3 + 1
gen_true = train - (line/15) ** 2 - .2
plt.plot(gen_true, label="generalization")
plt.ylabel("Accuracy")
plt.xlabel("Model Complexity")
plt.legend()
plt.savefig("images/overfitting_validation_set_1.svg")

In [ ]:
rng = np.random.RandomState(0)
plt.plot(train, label='training accuracy')
plt.plot(gen_true, label="generalization")
validation_set = gen_true + rng.normal(scale=.07, size=100)
plt.plot(validation_set, label="validation set")
plt.ylabel("Accuracy")
plt.xlabel("Model Complexity")
plt.legend()
plt.savefig("images/overfitting_validation_set_2.svg")

In [ ]:
import pandas as pd

In [ ]:
approval = pd.read_csv("data/approval_topline.csv", parse_dates=['timestamp'], index_col='timestamp')

In [ ]:
approval_est = approval.loc[approval['subgroup'] == "Adults", ["approve_estimate"]]

In [ ]:
approval_est.to_csv("data/approval_processed.csv")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
approval_est.plot(ax=ax)
ax.set_title("Approval Ratings")
fig.savefig("images/approval_ratings.png")

In [ ]:
from matplotlib.patches import Rectangle
rng = np.random.RandomState(42)
fig, ax = plt.subplots(figsize=(10, 6))
approval_est.plot(ax=ax)
xlim, ylim = ax.get_xlim(), ax.get_ylim()

for i in range(20):
    rect = Rectangle((rng.randint(xlim[0], xlim[1]), ylim[0]), 10,
                     ylim[1]-ylim[0], facecolor='#FFAAAA', alpha=0.8)
    ax.add_artist(rect)
    
# plt.title("Presidential approval estimates by fivethirtyeight")
plt.legend([rect], ['Random Test Set'] )
ax.set_xlabel("")
ax.set_title("Approval Ratings")
fig.savefig("images/approval_ratings_random.png")

In [ ]:
from matplotlib.patches import Rectangle

rng = np.random.RandomState(42)
approval_est.plot()

ax = plt.gca()
xlim, ylim = ax.get_xlim(), ax.get_ylim()
rect = Rectangle((xlim[1] - 300, ylim[0]), 300, ylim[1]-ylim[0], facecolor='#FFAAAA', alpha=0.8)
ax.add_artist(rect)

plt.title("Approvael Ratings")
plt.legend([rect], ['Structured Test Set'] )
ax.set_xlabel("")
plt.savefig("images/approval_ratings_structured.png")